In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, transform!, evaluate, ncoeff
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using Roots
using NLsolve
using Optim: InverseDiagonal#, ldiv!, dot

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


In [18]:
    # Dense rescaling
    Nx = 100
    Ne = 500
    ens = EnsembleState(Nx, Ne)

    ens.S .= randn(Nx).+ randn(Nx, Ne) .* randn(Nx, Ne)
    X = deepcopy(ens.S)
    L = LinearTransform(X; diag = true)

    AdaptiveTransportMap.transform!(L, X);

    @test norm(mean(X;dims = 2))<1e-10
    @test norm(  - ones(Nx))<1e-10

#     AdaptiveTransportMap.itransform!(L, X)

#     @test norm(mean(X;dims = 2) - mean(ens))<1e-10
#     @test norm(cov(X')  - cov(ens))<1e-10

Error During Test at In[18]:13
  Test threw exception
  Expression: norm(std(X'; dims = 2) - ones(Nx)) < 1.0e-10
  DimensionMismatch("dimensions must match: a has dims (Base.OneTo(500), Base.OneTo(1)), b has dims (Base.OneTo(100),), mismatch at 1")
  Stacktrace:
   [1] promote_shape at ./indices.jl:178 [inlined]
   [2] promote_shape at ./indices.jl:169 [inlined]
   [3] -(::Array{Float64,2}, ::Array{Float64,1}) at ./arraymath.jl:38
   [4] top-level scope at In[18]:13
   [5] eval at ./boot.jl:331 [inlined]
   [6] softscope_include_string(::Module, ::String, ::String) at /home/mat/.julia/packages/SoftGlobalScope/cSbw5/src/SoftGlobalScope.jl:218
   [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/mat/.julia/packages/IJulia/DrVMH/src/execute_request.jl:67
   [8] #invokelatest#1 at ./essentials.jl:712 [inlined]
   [9] invokelatest at ./essentials.jl:711 [inlined]
   [10] eventloop(::ZMQ.Socket) at /home/mat/.julia/packages/IJulia/DrVMH/src/eventloop.jl:8
   [11] (::IJulia.var"#15#18"

Test.FallbackTestSetException: There was an error during testing

In [19]:
std('; dims = 2)

500×1 Array{Float64,2}:
 0.8101215623773982
 1.162590278953438
 0.8406826135213554
 0.9024312887460753
 0.9107346623500261
 0.9546151445907091
 0.9071210811677319
 1.187432484267459
 1.0250801129223708
 0.9805963600090831
 1.0551075540041086
 1.1726245413799852
 0.7384640022543452
 ⋮
 1.0386652205108282
 0.8434050507745406
 0.8759666868957762
 0.9665224711918406
 0.7651309274662362
 1.2410904711733703
 0.9983748546166819
 0.9111262849973125
 1.1480084986213006
 0.7011055266686552
 0.963754894251547
 0.9518569861301202

In [8]:
?ldiv!

search: ldiv! splitdrive



```
ldiv!(Y, A, B) -> Y
```

Compute `A \ B` in-place and store the result in `Y`, returning the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = zero(X);

julia> ldiv!(Y, qr(A), X);

julia> Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(A, B)
```

Compute `A \ B` in-place and overwriting `B` to store the result.

The argument `A` should *not* be a matrix.  Rather, instead of matrices it should be a factorization object (e.g. produced by [`factorize`](@ref) or [`cholesky`](@ref)). The reason for this is that factorization itself is both expensive and typically allocates memory (although it can also be done in-place via, e.g., [`lu!`](@ref)), and performance-critical situations requiring `ldiv!` usually also require fine-grained control over the factorization of `A`.

# Examples

```jldoctest
julia> A = [1 2.2 4; 3.1 0.2 3; 4 1 2];

julia> X = [1; 2.5; 3];

julia> Y = copy(X);

julia> ldiv!(qr(A), X);

julia> X
3-element Array{Float64,1}:
  0.7128099173553719
 -0.051652892561983674
  0.10020661157024757

julia> A\Y
3-element Array{Float64,1}:
  0.7128099173553719
 -0.05165289256198333
  0.10020661157024785
```

---

```
ldiv!(a::Number, B::AbstractArray)
```

Divide each entry in an array `B` by a scalar `a` overwriting `B` in-place.  Use [`rdiv!`](@ref) to divide scalar from right.

# Examples

```jldoctest
julia> B = [1.0 2.0; 3.0 4.0]
2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

julia> ldiv!(2.0, B)
2×2 Array{Float64,2}:
 0.5  1.0
 1.5  2.0
```

---

Solve Hy = rhs for a non-square Hessenberg matrix. Note that `H` is also modified as is it converted to an upper triangular matrix via Given's rotations


In [32]:
typeof(cholesky(cov(ens)).L)<:LowerTriangular

true

search: copyto! unsafe_copyto! copy_transpose! FamilyScaledProPolyHermite



```
copyto!(dest, do, src, so, N)
```

Copy `N` elements from collection `src` starting at offset `so`, to array `dest` starting at offset `do`. Return `dest`.

---

```
copyto!(dest::AbstractArray, src) -> dest
```

Copy all elements from collection `src` to array `dest`, whose length must be greater than or equal to the length `n` of `src`. The first `n` elements of `dest` are overwritten, the other elements are left untouched.

# Examples

```jldoctest
julia> x = [1., 0., 3., 0., 5.];

julia> y = zeros(7);

julia> copyto!(y, x);

julia> y
7-element Array{Float64,1}:
 1.0
 0.0
 3.0
 0.0
 5.0
 0.0
 0.0
```

---

```
copyto!(dest, Rdest::CartesianIndices, src, Rsrc::CartesianIndices) -> dest
```

Copy the block of `src` in the range of `Rsrc` to the block of `dest` in the range of `Rdest`. The sizes of the two regions must match.

---

```
copyto!(dest::AbstractMatrix, src::UniformScaling)
```

Copies a [`UniformScaling`](@ref) onto a matrix.

!!! compat "Julia 1.1"
    In Julia 1.0 this method only supported a square destination matrix. Julia 1.1. added support for a rectangular matrix.



### Hessian computation 

In [22]:
Nx = 4
m = 10
B = MultiBasis(CstProHermite(m; scaled =true), Nx)

Nψ = 14
# idx = rand(0:5,Nψ, Nx)
idx =Matrix([5  2  5  3;
             0  2  2  5;
             5  4  3  4;
             4  1  3  1;
             3  2  4  5;
             0  3  3  3;
             4  2  0  0;
             2  0  2  4;
             4  4  0  5;
             0  4  0  2;
             4  5  5  5;
             0  1  2  3;
             4  2  4  0;
             4  3  2  3]);


coeff = [  2.421476831418713;
          0.23398851530507098;
         -0.5285368375369742;
          0.8947095708709891;
         -0.688462862924509;
          0.23331743780756214;
          2.0098809355359006;
          1.1243983698817437;
         -0.18373487759949592;
          0.24787268168991677;
          0.9041136183543212;
          1.9776434399606355;
          1.1972246121155632;
          0.3847306368698849]

f = ExpandedFunction(B, idx, coeff)

fp = ParametricFunction(f)
R = IntegratedFunction(fp)
H = HermiteMapk(R; α = 1e-6);
Ne = 500

500

In [23]:
sum(idx;dims = 2)

14×1 Array{Int64,2}:
 15
  9
 16
  9
 14
  9
  6
  8
 13
  6
 19
  6
 10
 12

In [24]:
X = randn(Nx, Ne) .* randn(Nx, Ne) 
S = Storage(H.I.f, X; hess = false);

J = 0.0
dJ = zeros(Nψ)
d2J = zeros(Nψ, Nψ)

@time negative_log_likelihood!(J, dJ, coeff, S, H, X)

precond!

  0.002007 seconds (1.53 k allocations: 2.742 MiB)


precond! (generic function with 2 methods)

In [25]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, Optim.LBFGS())

  104.624 ms (130849 allocations: 233.12 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.24e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.82e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.87e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.26e-13 ≰ 0.0e+00
    |g(x)|                 = 2.84e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    28
    f(x) calls:    85
    ∇f(x) calls:   85


In [26]:
struct Preconditioner
    P::Symmetric{Float64}
    
    F::Cholesky{Float64, Matrix{Float64}}
end

function Preconditioner(P::Matrix{Float64})
    return Preconditioner(Symmetric(P), cholesky(Symmetric(P)))
end

ldiv!(x, P::Preconditioner, b) = copyto!(x, P.F \ b)
dot(A::Array, P::Preconditioner, B::Vector) = dot(A, P.P, B)

ErrorException: error in method definition: function LinearAlgebra.ldiv! must be explicitly imported to be extended

In [27]:

# res = Optim.optimize(Optim.only_fg!(negative_loglikelihood!(S, H, X)), coeff, 
#             Optim.LBFGS(P = Preconditioner(precond)))#, precondprep = (P, x) -> precond!(S, H, X)(P, x)))

In [28]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, Optim.LBFGS())#, precondprep = (P, x) -> Preconditioner(precond!($S, $H, $X)($P.P.data, x))))

  104.817 ms (130849 allocations: 233.12 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.24e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.82e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.87e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.26e-13 ≰ 0.0e+00
    |g(x)|                 = 2.84e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    28
    f(x) calls:    85
    ∇f(x) calls:   85


In [29]:
precond = zeros(Nψ, Nψ)
coeff0 = randn(Nψ)
precond!(precond, coeff, S, H, X)
P = Preconditioner(precond)

res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), coeff0, Optim.LBFGS(; m = 20, P = Preconditioner(precond)))

 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 6.21e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.10e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 7.68e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.19e-13 ≰ 0.0e+00
    |g(x)|                 = 3.43e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [30]:
Optim.minimizer(res)

14-element Array{Float64,1}:
  0.2042411208812283
 -0.160665456718217
  5.371746068389059
 -0.19909085135432283
  6.822999267856531
 -0.14374191048398882
 -0.3861327468309733
  0.788767701800107
  2.22362622541736
 -0.9700837073808384
  2.7757966850015574
  0.04063103525990578
  1.7876303361723063
 -1.6264347964407893

In [31]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Preconditioner($precond), precondprep = (P, x) -> Preconditioner(precond!($S, $H, $X)($P.P.data, x))))

  72.319 ms (38554 allocations: 68.59 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [32]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(P = Preconditioner($precond)))#, precondprep = (P, x) -> Preconditioner(precond!($S, $H, $X)($P.P.data, x))))

  19.128 ms (26245 allocations: 46.64 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [33]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Preconditioner($precond)))#, precondprep = (P, x) -> Preconditioner(precond!(S, H, X)(P.P.data, x))))

  19.342 ms (26265 allocations: 46.64 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [34]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Preconditioner($precond)))

  19.304 ms (26265 allocations: 46.64 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 8.58e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.74e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.18e-12 ≰ 0.0e+00
    |g(x)|                 = 5.12e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    8
    f(x) calls:    17
    ∇f(x) calls:   17


In [39]:
diagprecond = zeros(Nψ)
coeff0 = randn(Nψ)
diagprecond!(diagprecond, coeff, S, H, X)
D = Optim.InverseDiagonal(diagprecond)

ldiv!(out::Array, P::Optim.InverseDiagonal, A::Array) = copyto!(out, A .* P.diag)
dot(A::Array, P::Optim.InverseDiagonal, B::Vector) = dot(A, B ./ P.diag)

dot (generic function with 40 methods)

In [41]:
diagprecond

14-element Array{Float64,1}:
 0.00012403916115351907
 0.000914061082337064
 4.223509631951479e-5
 0.0010497362861250537
 9.846338287295053e-5
 0.0023101801954228643
 0.006057967285600174
 0.000385561839624177
 0.0005285393135169043
 0.003055127424459309
 6.995783516676906e-5
 0.003976096151961689
 0.0004067998335964076
 0.000124219015787626

In [44]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), coeff, 
            Optim.LBFGS(; m = 20, P = Diagonal(diagprecond)))

 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 6.88e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.34e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.63e-13 ≰ 0.0e+00
    |g(x)|                 = 9.72e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    11
    f(x) calls:    23
    ∇f(x) calls:   23


In [45]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, 
            Optim.LBFGS(; m = 20, P = Diagonal($diagprecond)))

  25.826 ms (35493 allocations: 63.09 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 6.88e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.34e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.63e-13 ≰ 0.0e+00
    |g(x)|                 = 9.72e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    11
    f(x) calls:    23
    ∇f(x) calls:   23


In [37]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), zeros(Nψ), Optim.NGMRES())

 * Status: success

 * Candidate solution
    Final objective value:     1.479295e+00

 * Found with
    Algorithm:     Nonlinear GMRES preconditioned with Gradient Descent

 * Convergence measures
    |x - x'|               = 3.87e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 5.68e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.85e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.93e-13 ≰ 0.0e+00
    |g(x)|                 = 5.95e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    32
    f(x) calls:    123
    ∇f(x) calls:   123


In [128]:
@btime res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!($S, $H, $X)), $coeff, Optim.AcceleratedGradientDescent())

  3.519 s (3527117 allocations: 6.96 GiB)


 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     1.572975e+00

 * Found with
    Algorithm:     Accelerated Gradient Descent

 * Convergence measures
    |x - x'|               = 4.30e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.07e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.20e-09 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.40e-09 ≰ 0.0e+00
    |g(x)|                 = 1.35e-05 ≰ 1.0e-08

 * Work counters
    Seconds run:   4  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    3295
    ∇f(x) calls:   3295


In [ ]:
MomentumGradientDescent()

In [30]:
X = randn(Nx, Ne) .* randn(Nx, Ne) 
S = Storage(H.I.f, X);

J = 0.0
dJ = zeros(Nψ)
d2J = zeros(Nψ, Nψ)
hess_negative_log_likelihood!(J, dJ, d2J, coeff, S, H, X)

1.3784285990365932

In [117]:
dJ

11-element Array{Float64,1}:
 -0.647941708993688
  0.01751723817324542
  0.12943970119038775
  0.0033091844731214684
 -0.00031264752126695754
  0.03545495128594644
 -0.0062483204244866645
 -0.028569877746525366
 -0.0008423358431159828
 -0.0033284486942029698
 -0.005733695198669693

In [31]:
@btime res = Optim.optimize(Optim.only_fgh!(hess_negative_log_likelihood!($S, $H, $X)), $coeff, Optim.NewtonTrustRegion())
# coeffopt = Optim.minimizer(res)

  193.793 ms (82746 allocations: 143.66 MiB)


 * Status: success

 * Candidate solution
    Final objective value:     1.341428e+00

 * Found with
    Algorithm:     Newton's Method (Trust Region)

 * Convergence measures
    |x - x'|               = 1.16e-05 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.01e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.38e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.03e-13 ≰ 0.0e+00
    |g(x)|                 = 2.62e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    28
    f(x) calls:    29
    ∇f(x) calls:   29
    ∇²f(x) calls:  23


In [40]:
@time res = Optim.optimize(Optim.only_fgh!(hess_negative_log_likelihood!(S, H, X)), coeff, Optim.NewtonTrustRegion())

  0.025023 seconds (17.23 k allocations: 44.434 MiB, 25.59% gc time)


 * Status: success

 * Candidate solution
    Final objective value:     1.450904e+00

 * Found with
    Algorithm:     Newton's Method (Trust Region)

 * Convergence measures
    |x - x'|               = 3.17e-04 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.29e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.02e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 7.03e-12 ≰ 0.0e+00
    |g(x)|                 = 1.89e-13 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    4
    f(x) calls:    5
    ∇f(x) calls:   5
    ∇²f(x) calls:  4


In [121]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), coeff, Optim.LBFGS())
# coeffopt = Optim.minimizer(res)

 * Status: success

 * Candidate solution
    Final objective value:     1.298813e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.47e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.06e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.15e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.97e-14 ≰ 0.0e+00
    |g(x)|                 = 6.45e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    19
    f(x) calls:    58
    ∇f(x) calls:   58


In [68]:
@time negative_log_likelihood!(J, dJ, coeff, S, H, X)

  0.000169 seconds (793 allocations: 47.328 KiB)


2.137129544913297

### Greedy fit

In [147]:
Nx = 10

m = 20

Ne = 1000
# ens = EnsembleState(Nx, Ne)

# ens.S .=  [0.267333   1.43021;
#           0.364979   0.607224;
#          -1.23693    0.249277;
#          -2.0526     0.915629;
#          -0.182465   0.415874;
#           0.412907   1.01672;
#           1.41332   -0.918205;
#           0.766647  -1.00445]';
X = ens.S
X = randn(Nx, Ne).^2 + 0.1*randn(Nx, Ne)
# idx = [0 0; 0 1; 1 0; 0 2; 2 0; 1 1; 2 1; 1 2; 1 3; 2 2]

# Nψ = 10

# coeff = [     1.0174962905416995;
#              -1.1871356902490515;
#              -1.9837596947482932;
#               0.5088563113730136;
#              -0.9948770215816068;
#               1.1697089852843865;
#              -0.4364897348115606;
#              -0.45195264571030846;
#               0.6717865772395687;
#               0.8540132401727857]

In [148]:
L = LinearTransform(X; diag = true)
transform!(L, X)

10×1000 Array{Float64,2}:
 -0.690202   -0.595004    1.53707   …  -0.591998    -0.578563   1.52853
 -0.441275   -0.314905   -0.282054     -0.30521     -0.622784   0.690694
  1.15504    -0.652401   -0.618832     -0.0466972   -0.4555    -0.691359
  3.07661    -0.686147   -0.726459     -0.764043     0.390474   0.0371078
 -0.323856   -0.237024    0.167072     -0.611194    -0.624197   3.35201
  1.53622    -0.640471   -0.284045  …  -0.0834316   -0.310722  -0.291826
 -0.131834    0.0363916   0.112176     -0.601238    -0.554947  -0.451961
 -0.352505    1.71733    -0.753808      0.469905    -0.345014  -0.141933
 -0.0696934   0.0338294  -0.336468     -0.00743629   1.11219   -0.688029
 -0.452564   -0.711301   -0.498921     -0.512323     0.983083  -0.73837

In [149]:
X_train = X[:,1:800]
X_valid = X[:,801:1000]

10×200 Array{Float64,2}:
 -0.733241  -0.588017   -0.570597   …  -0.591998    -0.578563   1.52853
 -0.504441   2.32238    -0.669967      -0.30521     -0.622784   0.690694
  0.63604   -0.596369   -0.220347      -0.0466972   -0.4555    -0.691359
 -0.428104  -0.767242   -0.0572687     -0.764043     0.390474   0.0371078
  0.287644  -0.37097     0.0700383     -0.611194    -0.624197   3.35201
 -0.102864   0.0141409  -0.687785   …  -0.0834316   -0.310722  -0.291826
 -0.232743  -0.752499   -0.475897      -0.601238    -0.554947  -0.451961
 -0.513879  -0.585413    0.42345        0.469905    -0.345014  -0.141933
 -0.400458  -0.638978   -0.599365      -0.00743629   1.11219   -0.688029
 -0.52674   -0.414987    2.33183       -0.512323     0.983083  -0.73837

In [152]:
@code_warntype greedyfit(m, Nx, X_train, X_valid, 15; verbose = false);

Variables
  #unused#::Core.Compiler.Const(AdaptiveTransportMap.var"#greedyfit##kw"(), false)
  @_2::NamedTuple{(:verbose,),Tuple{Bool}}
  @_3::Core.Compiler.Const(AdaptiveTransportMap.greedyfit, false)
  m::Int64
  k::Int64
  X::Array{Float64,2}
  Xvalid::Array{Float64,2}
  maxterms::Int64
  maxpatience::Int64
  verbose::Bool
  @_11::Int64
  @_12::Bool

Body::Tuple{HermiteMapk{_A,_B,_C} where _C where _B where _A,Array{Float64,1},Array{Float64,1}}
1 ─ %1  = Base.haskey(@_2, :maxpatience)::Core.Compiler.Const(false, false)
└──       goto #3 if not %1
2 ─       Core.Compiler.Const(:(Base.getindex(@_2, :maxpatience)), false)
│         Core.Compiler.Const(:(%3 isa AdaptiveTransportMap.Int64), false)
│         Core.Compiler.Const(:(%4), false)
│         Core.Compiler.Const(:(goto %9), false)
│         Core.Compiler.Const(:(%new(Core.TypeError, Symbol("keyword argument"), :maxpatience, AdaptiveTransportMap.Int64, %3)), false)
│         Core.Compiler.Const(:(Core.throw(%7)), false)
│         

In [151]:
@code_warntype greedyfit($m, $Nx, $X_train, $X_valid, $15; verbose = false);

ErrorException: syntax: "$" expression outside quote

In [89]:
@btime greedyfit($m, $Nx, $X_train, $X_valid, $10; verbose = false);

  1.001 s (1279836 allocations: 2.03 GiB)


In [99]:
Hk_old = HermiteMapk(m, Nx; α = 1e-6);

@time S = Storage(Hk_old.I.f, X);

Hk_new, train_error, valid_error = greedyfit(m, Nx, X_train, X_valid, 10; verbose = true);

  0.000038 seconds (42 allocations: 120.516 KiB)
1 terms - Training error: 1.4443504670380543, Validation error: 1.3147907978711455
2 terms - Training error: 1.013555754394048, Validation error: 1.0148284980423214
3 terms - Training error: 0.9740035731615574, Validation error: 0.9536709077333043
4 terms - Training error: 0.968194963388136, Validation error: 0.9269258934472301
5 terms - Training error: 0.8660654654774056, Validation error: 0.8538651976765612
6 terms - Training error: 0.8468938234481649, Validation error: 0.8374329264723278
7 terms - Training error: 0.8118015219715872, Validation error: 0.8007373200002506
8 terms - Training error: 0.808988833791454, Validation error: 0.8016700608672841
9 terms - Training error: 0.8055509920369043, Validation error: 0.8064967607768131
10 terms - Training error: 0.7857644650326624, Validation error: 0.8090534466320567


In [106]:
Hk_old = HermiteMapk(m, Nx; α = 1e-6);

@time S = Storage(Hk_old.I.f, X);

Hk_new, train_error, valid_error = greedyfit(m, Nx, X_train, X_valid, 10; verbose = true);

  0.000057 seconds (42 allocations: 120.516 KiB)
1 terms - Training error: 1.41428818328749, Validation error: 1.435039932873401
2 terms - Training error: 1.011647231546337, Validation error: 1.0765340323252046
3 terms - Training error: 0.954187562314565, Validation error: 1.009496573370243
4 terms - Training error: 0.9532824496069529, Validation error: 1.007776307368724
5 terms - Training error: 0.841380513349966, Validation error: 0.9041126563257862
6 terms - Training error: 0.8299467025145106, Validation error: 0.9037778557535633
7 terms - Training error: 0.8018842783083129, Validation error: 0.8750517929963674
8 terms - Training error: 0.801758584142784, Validation error: 0.8767880843281514
9 terms - Training error: 0.8015539210804201, Validation error: 0.8774101595696208
10 terms - Training error: 0.7995583928098936, Validation error: 0.8823916723449652


In [83]:
Hk_test = deepcopy(Hk_new)
setcoeff!(Hk_test, zero(getcoeff(Hk_new)));

In [85]:
S_test = Storage(Hk_test.I.f, X_train)
coeff_test = getcoeff(Hk_test)

res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S_test, Hk_test, X_train)), coeff_test, Optim.BFGS())
Optim.minimizer(res)

10-element Array{Float64,1}:
 -11.217886066730607
  11.50914577979369
  26.61111083798624
 -24.71937861667548
  34.997147393519235
 -17.03501613771925
  11.077321531464895
   0.034843907277600714
   0.0381866111200113
   0.1782552001411759

In [78]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, Hk_old, ens.S)), coeff, Optim.BFGS())
coeffopt = Optim.minimizer(res)

1-element Array{Float64,1}:
 -1.144117388080984